<a href="https://colab.research.google.com/github/Meridor6919/BeeClassification/blob/master/HighDropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import keras
from keras import layers
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras import datasets, layers, models, losses
import matplotlib.pyplot as plt

In [2]:
dataset = tfds.load('bee_dataset', split=['train'], as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/7490 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/bee_dataset/bee_dataset_300/1.0.0.incompleteCNC0CH/bee_dataset-train.tfrecord*…

Dataset bee_dataset downloaded and prepared to ~/tensorflow_datasets/bee_dataset/bee_dataset_300/1.0.0. Subsequent calls will reuse this data.


In [3]:
#Split between training data, validation data, test data
splitSize = (0.0, 0.765, 0.9, 1.0)

Testing precision on red channel only

In [4]:
array = np.vstack(tfds.as_numpy(dataset[0]))
ranges = list([int(array.shape[0] * x) for x in splitSize])

def getDataset(ranges, idx):
    return (
        np.array(list(map(lambda x: x[0][:, :, :], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['varroa_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['pollen_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['wasps_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['cooling_output'], array[ranges[idx]: ranges[idx + 1]])))
    )
train_images, train_labels1,train_labels2,train_labels3, train_labels4= getDataset(ranges, 0)
validation_images, validation_labels1,validation_labels2, validation_labels3, validation_labels4 = getDataset(ranges, 1)
test_images, test_labels1,test_labels2,test_labels3, test_labels4  = getDataset(ranges, 2)


<ipython-input-4-e43ac078c997>:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  array = np.vstack(tfds.as_numpy(dataset[0]))
/usr/local/lib/python3.8/dist-packages/numpy/core/shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


In [18]:
train_labels = np.dstack((train_labels1, train_labels2, train_labels3,train_labels4)).squeeze()
validation_labels = np.dstack((validation_labels1,validation_labels2, validation_labels3, validation_labels4)).squeeze()
test_labels = np.dstack((test_labels1,test_labels2,test_labels3, test_labels4)).squeeze()

In [6]:
class my_metric:
  binaryAcc = tf.keras.metrics.BinaryAccuracy(threshold=0.9)

  def all(y_true, y_pred):
    my_metric.binaryAcc.reset_state()
    my_metric.binaryAcc.update_state(y_true, y_pred)
    return my_metric.binaryAcc.result()

  def varroa(y_true, y_pred):
    my_metric.binaryAcc.reset_state()
    my_metric.binaryAcc.update_state(y_true[:, 0], y_pred[:, 0])
    return my_metric.binaryAcc.result()

  def pollen(y_true, y_pred):
    my_metric.binaryAcc.reset_state()
    my_metric.binaryAcc.update_state(y_true[:, 1], y_pred[:, 1])
    return my_metric.binaryAcc.result()

  def wasps(y_true, y_pred):
    my_metric.binaryAcc.reset_state()
    my_metric.binaryAcc.update_state(y_true[:, 2], y_pred[:, 2])
    return my_metric.binaryAcc.result()

  def cooling(y_true, y_pred):
    my_metric.binaryAcc.reset_state()
    my_metric.binaryAcc.update_state(y_true[:, 3], y_pred[:, 3])
    return my_metric.binaryAcc.result()

In [14]:
import datetime
import tensorflow_hub as hub
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(log_dir)
  return tensorboard_callback
     

In [20]:
model= tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=10, 
                         kernel_size=3, 
                         activation="relu", 
                         input_shape=(300, 150, 3)), 
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, 
                            padding="valid"), 
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"), 

  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(4, activation="sigmoid") 
])

model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=[my_metric.all, my_metric.varroa, my_metric.pollen, my_metric.wasps, my_metric.cooling,])

history = model.fit(train_images, train_labels,
                        epochs=20,
                        batch_size = 32,
                        callbacks = create_tensorboard_callback(dir_name="tensorflow_hub", 
                                                                                     experiment_name="model"),
                        validation_data=(validation_images, validation_labels))

tensorflow_hub/model/20230110-131824
Epoch 1/20
180/180 [==============================] - 9s 42ms/step - loss: 1.9922 - all: 0.8727 - varroa: 0.8233 - pollen: 0.8575 - wasps: 0.9314 - cooling: 0.8788 - val_loss: 0.1846 - val_all: 0.9012 - val_varroa: 0.8285 - val_pollen: 0.8508 - val_wasps: 0.9883 - val_cooling: 0.9373
Epoch 2/20
180/180 [==============================] - 7s 39ms/step - loss: 0.1504 - all: 0.9180 - varroa: 0.8503 - pollen: 0.8762 - wasps: 0.9856 - cooling: 0.9597 - val_loss: 0.1549 - val_all: 0.9116 - val_varroa: 0.8418 - val_pollen: 0.8551 - val_wasps: 0.9941 - val_cooling: 0.9555
Epoch 3/20
180/180 [==============================] - 8s 43ms/step - loss: 0.1113 - all: 0.9332 - varroa: 0.8714 - pollen: 0.8960 - wasps: 0.9925 - cooling: 0.9727 - val_loss: 0.1579 - val_all: 0.9184 - val_varroa: 0.8350 - val_pollen: 0.8684 - val_wasps: 0.9951 - val_cooling: 0.9750
Epoch 4/20
180/180 [==============================] - 8s 43ms/step - loss: 0.0911 - all: 0.9421 - varroa: 0.

In [21]:
!tensorboard dev upload --logdir ./tensorflow_hub/ \
  --name "model" \
  --description "dropout layer 0.5" \
  --one_shot


New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/DXPQnEI4TxGAs4C0gzg7rQ/

[2023-01-10T13:21:52] Started scanning logdir.
[2023-01-10T13:21:55] Total uploaded: 720 scalars, 0 tensors, 2 binary objects (159.3 kB)
[2023-01-10T13:21:55] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/DXPQnEI4TxGAs4C0gzg7rQ/


In [22]:
model.evaluate(test_images, test_labels)

24/24 [==============================] - 0s 19ms/step - loss: 0.1835 - all: 0.9496 - varroa: 0.9043 - pollen: 0.9252 - wasps: 0.9844 - cooling: 0.9844


[0.18354351818561554,
 0.9495692849159241,
 0.9043469429016113,
 0.9251802563667297,
 0.984375,
 0.984375]

In [ ]:
x = test_images[555]
plt.imshow(x)

In [ ]:
model.predict(x.reshape(1,300,150,3)), test_labels[555]

In [ ]:
x = test_images[144]
plt.imshow(x)

In [ ]:
model.predict(x.reshape(1,300,150,3)), test_labels[144]